In [48]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from scipy.optimize import nnls

In [59]:
def jacobi(A, b, x0, nit):    
    """ Método de Jacobi para solução do sistema Ax=b.

    x = jacobi(A, b, x0, nit)

    Entradas:
        - A: matriz n x n (tipo: lista de listas)
        - b: vetor n x 1 (tipo: lista)
        - x0: vetor n x 1, ponto de partida (tipo: lista)
        - nit: número de iterações
    
    Saídas:
        - x:  matriz n x nit, contendo a sequência de vetores
        - c:  norma da matriz de iteração
    """ 
    
    import numpy as np    
    
    f = lambda obj: isinstance(obj,list)     
    if not all([f(A),f(b),f(x0)]):
        raise TypeError('A, b e x0 devem ser do tipo list.')
    else:
        A = np.asarray(A)
        b = np.asarray(b)
        x0 = np.asarray(x0)
            
    n = np.size(b)
    x = np.zeros((n, nit),dtype=float)
    x[:,0] = x0
        
    P = np.diag(np.diag(A))
    Q = P - A # elementos de fora da diagonal
    C = np.linalg.solve(P,Q) # matriz da função de iteração
    g = np.linalg.solve(P,b) # vetor da função de iteração
    c = np.linalg.norm(C) 
    
    #processo iterativo
    X = x0[:]    
    j = 1
    for j in range(1,nit):        
        X = C.dot(X) + g        
        x[:,j] = X
        
    print(f"Vetor solução:{x[:,-1]}")                
    print(f"||C|| = {c:.4f}")                
        
    return x, c

In [63]:
def gauss_seidel(A, b, x0, nit, tol):
    """ Método de Gauss-Seidel de solucao do sistema Ax=b

    Entradas:
        - A: matriz n x n (tipo: lista de listas)
        - b: vetor n x 1 (tipo: lista)
        - x0: vetor n x 1, ponto de partida (tipo: lista)
        - nit: número de iterações
        - tol: tolerancia de erro relativo
   
    Saídas:
        - x: matriz n x N, contendo a sequência de vetores
        - c: norma da matriz de iteração       
        - err: erro relativo percentual por iteracao (norma 2)
    """

    import numpy as np
    n = len(b)
    x = np.zeros((n, nit))
    x[:,0] = x0
    xx = x0 # iterada atual
    P = np.tril(A)
    N = P - A
    C = np.linalg.solve(P, N)
    c = np.linalg.norm(C)
    g = np.linalg.solve(P, b)
    
    
    for j in range(1, nit):
        xx = C @ xx + g
        x[:,j] = xx
         
    key = True    
        
    for i in range(1, nit):   
             
        e = np.linalg.norm(x[:,i] - x[:,i-1])/np.linalg.norm(x[:,i])
            
        if (e < tol) and (key == True):                        
            key = False

    print(f'Vetor-solução: {x[:,-1]}')
    print(f"||C|| = {c:.4f}")
    
    return x, c


In [50]:
# Subsystems x Sources
subsystems = ["Shield", "Defence", "Fighter", "IonCannon", "Engines"]
sources = ["FusionCells", "HyperCores", "SolarCaps", "Geothermal", "Batteries"] # podem substituir alguma dessas últimas pelos ratos

´´
#Achei que fazia mais sentido colocar quantos % de cada fonte eles vão pegar, ao invés de quantidade, mas podem mudar isso, não vai alterar o funcionamento
E = np.array([
    [0.4, 0.7, 0.6, 0.5, 0.0],   # Shield
    [0.3, 0.8, 0.2, 0.4, 0.5],   # Defence
    [0.5, 0.9, 0.3, 0.6, 0.4],   # Fighter
    [0.6, 0.7, 0.2, 0.5, 0.3],   # IonCannon
    [0.4, 0.6, 0.7, 0.8, 0.5],   # Engines
], dtype=float)
´´´

In [52]:
E = [[25, 7, 6, 5, 0], [3,22,2,4,5], [5,9,30,6,4], [6,7,2,28,3], [4,6,7,8,35]]

In [53]:
#Quantos de cada subsistemas existem
d = np.array([50, 120, 200, 100, 90], dtype=float)

In [54]:
#LU
#Caderno 8

# Consideraremos que as matrizes triangulares inferiores sempre terão a sua diagonal principal formada por entradas iguais a 1.
# L (triangular inferior) e U (triangular superior)
# piv: o vetor de pivoteamento
lu, piv = lu_factor(E) # Fatoração LU
x_lu = lu_solve((lu, piv), d) #Ex = d
res_lu = np.linalg.norm(E @ x_lu - d) # Calcula o resíduo do sistema: r = Ex−d

In [55]:
x_lu

array([-0.92680326,  4.64125266,  4.93382293,  2.21567507,  0.38850103])

In [56]:
res_lu

np.float64(0.0)

In [61]:
B = [[25, 7, 6, 5, 0], [3,22,2,4,5], [5,9,30,6,4], [6,7,2,28,3], [4,6,7,8,35]]

b = [50, 120, 200, 100, 90]

x_jacobi = jacobi(B, b, [1, 1, 1, 1, 1], 150)

Vetor solução:[-0.92680326  4.64125266  4.93382293  2.21567507  0.38850103]
||C|| = 0.8502


In [67]:
V = [[25, 7, 6, 5, 0], [3,22,2,4,5], [5,9,30,6,4], [6,7,2,28,3], [4,6,7,8,35]]

v = [50, 120, 200, 100, 90]

x_gs = gauss_seidel(V, v, [1, 1, 1, 1, 1], 150, 1e-10)

Vetor-solução: [-0.92680326  4.64125266  4.93382293  2.21567507  0.38850103]
||C|| = 0.5503


In [66]:
x_gs

(array([[ 1.        ,  1.28      , -0.79509714, -0.96719984, -0.93735763,
         -0.92742458, -0.92660695, -0.92675268, -0.92680038, -0.92680422,
         -0.9268035 , -0.92680327, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.92680326, -0.92680326, -0.92680326,
         -0.92680326, -0.92680326, -0.